In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

## Import Dataframes

In [ ]:
#redacted

In [ ]:
patients = list(cnum_clinic["subject_id"].unique())

#subsetting by LCL plaquing data, but I think I'll actually need the #redacted data too
plaquing_justLCL = plaquing[plaquing.source == "LCL"]
missing_LCL_patients = [x for x in patients if x not in plaquing_justLCL["subject_id"].unique()]
print(missing_LCL_patients)


In [ ]:
columns_to_drop = ["sample_type", "study_id", "sp_id", "organism_id", "category", "Owner", "Tags", "Genus.species.", 
                   "Color", "Mucoid", 'BioSafety.Level','Town.City', 'State', 'Country', 'Body.Site.Organ', 'Medical.Diagnosis', 'Locus.program.']
cnum_clinic_subset = cnum_clinic.drop(columns = columns_to_drop)
cnum_clinic_subset.rename(columns={"grp_id" : "sample_id"}, inplace=True)

#Make trimmed clinc df a csv
cnum_clinic_subset.to_csv(#redacted, index=False)

#merge in the plaquing data
cnum_wPlaquing = pd.merge(cnum_clinic_subset, plaquing, on = ["subject_id", "sample_id", "visit", "day"], how = "left")

#Renaming some columns, getting a csv, making C numnbers capital
cnum_wPlaquing.rename(columns={"source_y": "plaquing_source", "subject_id" : "Patient", "Name.." : "Strain" }, inplace=True)
cnum_wPlaquing["Strain"] = cnum_wPlaquing["Strain"].str.upper()
cnum_wPlaquing.to_csv(#redacted", index=False)

In [ ]:
#losing some strains here, need to find missing data

missing_strains = [x for x in cnum_wPlaquing["Strain"].unique() if x not in cnum_just_titers["Strain"].unique()]

missing_strain_df = cnum_wPlaquing[cnum_wPlaquing["Strain"].isin(missing_strains)]
missing_sampleIDs = list(missing_strain_df["sample_id"].unique())

#Get missing data from LIMS and merge it to the cnum_wPlaquing df
lims_plaquing_data = pd.read_csv(#redacted)
cols_keep = ["Sample Description", "Result Name", "Result Formatted Entry", "Result Units"]
lims_data_subset = lims_plaquing_data[cols_keep]
lims_data_subset = lims_data_subset[lims_data_subset["Sample Description"].isin(missing_sampleIDs)]
print(lims_data_subset)
lims_data_subset = lims_data_subset.rename(columns = {"Sample Description" : "sample_id", "Result Name" : "test", "Result Formatted Entry" : "result", "Result Units" : "units"})

#merge it in
Allcnum_wPlaquing = pd.merge(cnum_wPlaquing, lims_data_subset, on=["sample_id"], how="left")
Allcnum_wPlaquing.to_csv(#redacted, index=False)
# ^Going to adjust this .csv manually and then bring it back in

In [ ]:
#Bringing in a manually adjusted .csv
plaquing_df = pd.read_csv(#redacted)

plaquing_df_titers = plaquing_df[(plaquing_df["units"] == "PFU_ML") | (plaquing_df["units"] == "(PFU/mL)")]
plaquing_df_titers.to_csv(#redacted, index=False)

#Going to manually make everything a number and then bring it back in
plaquing_titers = pd.read_csv(#redacted)

In [ ]:
#(max of 6 phage, avg of 6 phage), (titer, EOP), (all isos, unique isos, baseline isos)

#Get the max titer of each phage
#Checked, no cocktail was the max titer

max_titer_indices = plaquing_titers.groupby("Strain")["result"].idxmax()
cnum_maxTiter = plaquing_titers.loc[max_titer_indices]
#print(cnum_maxTiter["test"].value_counts())

#remove the cocktail points and find the average titer for the six phage
plaquing_titer_no_cock = plaquing_titers[plaquing_titers["test"] != "Cocktail/ avg titer (PFU/mL)"]
plaquing_titer_no_cock["test"].unique()

avg_strain_titer = plaquing_titer_no_cock.groupby("Strain")["result"].agg(["mean", "std"]).reset_index()
avg_strain_titer.columns = ["Strain", "Avg_Titer", "StDev"]

sorted_titers = avg_strain_titer.sort_values(by="Avg_Titer")

#convert to scientific notation
columns_to_convert = ['Avg_Titer', 'StDev']
sorted_titers[columns_to_convert] = sorted_titers[columns_to_convert].apply(lambda x: ['{:e}'.format(val) for val in x])
sorted_titers.to_csv("ClinIsolates_avg_Titers.csv", index=False)

#Genetically unique isolates
uniqueIso_titers = pd.merge(unique_isos, plaquing_titer_no_cock, on="Strain", how="left")

#Genetically Unique Iso max titer
uniqueIso_maxTiter_indices = uniqueIso_titers.groupby("Strain")["result"].idxmax()
uniqueIso_maxTiter = uniqueIso_titers.loc[uniqueIso_maxTiter_indices]

#Unique Iso avg titer
avg_uniqueIso_titer = uniqueIso_titers.groupby("Strain")["result"].agg(["mean", "std"]).reset_index()
avg_uniqueIso_titer.columns = ["Strain", "Avg_Titer", "StDev"]

sorted_unique_titers = avg_uniqueIso_titer.sort_values(by="Avg_Titer")
sorted_unique_titers[columns_to_convert] = sorted_unique_titers[columns_to_convert].apply(lambda x: ['{:e}'.format(val) for val in x])
sorted_unique_titers.to_csv("uniqueIsos_avg_Titers.csv", index=False)

#Baseline Isolates
baselineIso_titers = pd.merge(baseline_isos, plaquing_titer_no_cock, on=["Strain", "Patient"], how="left")

#Baseline Iso max titer
baselineIso_maxTiter_indices = baselineIso_titers.groupby(["Strain", "Patient"])["result"].idxmax()
baselineIso_maxTiter = baselineIso_titers.loc[baselineIso_maxTiter_indices]

#Baseline Iso avg titer
avg_baselineIso_titer = baselineIso_titers.groupby(["Strain", "Patient"])["result"].agg(["mean", "std"]).reset_index()
avg_baselineIso_titer.columns = ["Strain", "Patient", "Avg_Titer", "StDev"]

sorted_baseline_titers = avg_baselineIso_titer.sort_values(by="Avg_Titer")
sorted_baseline_titers[columns_to_convert] = sorted_baseline_titers[columns_to_convert].apply(lambda x: ['{:e}'.format(val) for val in x])
sorted_baseline_titers.to_csv(#redacted, index=False)

In [ ]:
#merge max titer with average titer
all_isos_maxTiter = cnum_maxTiter[["Strain", "result"]]
all_isos_maxTiter = all_isos_maxTiter.rename(columns = {"result" : "max_titer"})

ClinIsolates_plaquing = pd.merge(avg_strain_titer, all_isos_maxTiter, on="Strain", how="left")
ClinIsolates_plaquing_sorted = ClinIsolates_plaquing.sort_values(by="Avg_Titer")
new_columns_to_convert = ['Avg_Titer', 'StDev', "max_titer"]
ClinIsolates_plaquing_sorted[new_columns_to_convert] = ClinIsolates_plaquing_sorted[new_columns_to_convert].apply(lambda x: ['{:e}'.format(val) for val in x])

ClinIsolates_plaquing_sorted.to_csv(#redacted, index=False)

#Genetically unique isolates
unique_isos_plaquing = pd.merge(unique_isos, ClinIsolates_plaquing, on="Strain", how="left")
unique_isos_plaquing_sorted = unique_isos_plaquing.sort_values(by="Avg_Titer")
unique_isos_plaquing_sorted[new_columns_to_convert] = unique_isos_plaquing_sorted[new_columns_to_convert].apply(lambda x: ['{:e}'.format(val) for val in x])
unique_isos_plaquing_sorted.to_csv(#redacted, index=False)

#Baseline isolates
baseline_isos_plaquing = pd.merge(baseline_isos, ClinIsolates_plaquing, on="Strain", how="left")
baseline_isos_plaquing_sorted = baseline_isos_plaquing.sort_values(by="Avg_Titer")
baseline_isos_plaquing_sorted[new_columns_to_convert] = baseline_isos_plaquing_sorted[new_columns_to_convert].apply(lambda x: ['{:e}'.format(val) for val in x])
baseline_isos_plaquing_sorted.to_csv(#redacted, index=False)

#### EOP

In [ ]:
#EOP
plaquing_EOP = plaquing_df[plaquing_df["test"].str.contains("EOP")]
plaquing_EOP["result"] = pd.to_numeric(plaquing_EOP["result"])

#max EOP
max_EOP_indices = plaquing_EOP.groupby("Strain")["result"].idxmax()
cnum_maxEOP = plaquing_EOP.loc[max_EOP_indices]
all_isos_maxEOP = cnum_maxEOP[["Strain", "result"]]

#average EOP
avg_strain_EOP = plaquing_EOP.groupby("Strain")["result"].agg(["mean", "std"]).reset_index()
avg_strain_EOP.columns = ["Strain", "Avg_EOP", "StDev"]
ClinIsolates_EOP = pd.merge(avg_strain_EOP, all_isos_maxEOP, on='Strain', how='left')
ClinIsolates_EOP = ClinIsolates_EOP.rename(columns = {"result" : "max_EOP"})
ClinIsolates_EOP_sorted = ClinIsolates_EOP.sort_values(by="Avg_EOP")
ClinIsolates_EOP_sorted.to_csv(#redacted, index=False)

#Genetically unique isolates
unique_isos_EOP = pd.merge(unique_isos, ClinIsolates_EOP, on="Strain", how="left")
unique_isos_EOP_sorted = unique_isos_EOP.sort_values(by="Avg_EOP")
unique_isos_EOP_sorted.to_csv(#redacted, index=False)

#Baseline isolates
baseline_isos_EOP = pd.merge(baseline_isos, ClinIsolates_EOP, on="Strain", how="left")
baseline_isos_EOP_sorted = baseline_isos_EOP.sort_values(by="Avg_EOP")
baseline_isos_EOP_sorted.to_csv(#redacted, index=False)

#### Merge titers and EOPS

In [ ]:
ClinIsos_spots = pd.merge(ClinIsolates_plaquing_sorted, ClinIsolates_EOP_sorted, on="Strain", how="left")
ClinIsos_spots = ClinIsos_spots.rename(columns = {"StDev_x" : "StDev_Titer", "StDev_y" : "StDev_EOP"})
ClinIsos_spots.to_csv(#redacted, index=False)

unique_isos_spots = pd.merge(sorted_unique_titers, unique_isos_EOP_sorted, on="Strain", how="left")
unique_isos_spots = unique_isos_spots.rename(columns = {"StDev_x" : "StDev_Titer", "StDev_y" : "StDev_EOP"})
unique_isos_spots = unique_isos_spots[["Strain", "Patient", "Avg_Titer", "StDev_Titer", "Avg_EOP", "StDev_EOP", "max_EOP"]]
unique_isos_spots.to_csv(#redacted, index=False)

baseline_isos_spots = pd.merge(sorted_baseline_titers, baseline_isos_EOP_sorted, on=["Strain", "Patient"], how="left")
baseline_isos_spots = baseline_isos_spots.rename(columns = {"StDev_x" : "StDev_Titer", "StDev_y" : "StDev_EOP"})
baseline_isos_spots.to_csv(#redacted, index=False)

### Obtain Titers from ZOCs
##### Assuming a ZOC = 50 plaques

In [ ]:
ZOC_df = plaquing_df[plaquing_df["test"].str.contains("ZOC")]
ZOC_df["result"] = ZOC_df["result"].astype(float)
ZOC_df["ZOC_Titer"] = ((50 * (1/(ZOC_df["result"])))/.002)
ZOC_df["ZOC_Titer"] = ZOC_df["ZOC_Titer"].replace([np.inf, -np.inf], 0)
ZOC_df.to_csv("ZOC_df.csv", index=False)

#merge to big dataframe
ZOC_merge = ZOC_df[["Strain", "test", "ZOC_Titer"]]
# ZOC_merge = ZOC_merge.groupby("Strain")["ZOC_Titer"].agg(["Avg_ZOC_Titer", "std"]).reset_index()
# ZOC_merge
# clinIsoswZOC = pd.merge(ClinIsos_spots, ZOC_merge, on="Strain", how="left")
# clinIsoswZOC
print(len(ZOC_merge.Strain.unique()))

In [ ]:
print((1/ZOC_df["result"]).max())
print(((50 * (1e6)))/.002 / 1e10)

### sub in ZOC titers where there are ZOC titers

In [ ]:
plaquing_wZOC = pd.merge(plaquing_df, ZOC_merge, on=["Strain", "test"], how="left")

def ZOCSWAP(row):
    if row["ZOC_Titer"] > 0:
        row["result"] = row["ZOC_Titer"]
        row["units"] = "PFU_ML"
    return row

def ZOCDROP(row):
    rows_to_drop = []
    if row["ZOC_Titer"] > 0:
        rows_to_drop.append(row.name + 1)
    return rows_to_drop

getZOCd = plaquing_wZOC.apply(ZOCDROP, axis=1)

rows_to_drop_list = getZOCd.sum()

plaquing_ZOCSWAP = plaquing_wZOC.apply(ZOCSWAP, axis=1)
plaquing_ZOCSWAP = plaquing_ZOCSWAP.drop(rows_to_drop_list)
plaquing_ZOCSWAP.to_csv(#redacted, index=False)

#manually make them bumbers like last time
plaquing_ZOCSWAP_asnums = pd.read_csv(#redacted)

### Do what I did before with ZOCs Swapped

In [ ]:
plaquing_ZOCSWAP_titers = plaquing_ZOCSWAP_asnums[(plaquing_ZOCSWAP_asnums["units"] == "PFU_ML") | (plaquing_ZOCSWAP_asnums["units"] == "(PFU/mL)")]
ZOC_no_cock = plaquing_ZOCSWAP_titers[plaquing_ZOCSWAP_titers["test"] != "Cocktail/ avg titer (PFU/mL)"]
ZOC_no_cock['result'] = ZOC_no_cock['result'].astype(float)

#max titer with ZOC
maxZOCindices = ZOC_no_cock.groupby("Strain")["result"].idxmax()
max_titer_wZOC = ZOC_no_cock.loc[maxZOCindices]
ZOCmax_whatmatters = max_titer_wZOC[["Strain", "result"]]
ZOCmax_whatmatters = ZOCmax_whatmatters.rename(columns = {"result" : "max_titer_wZOC"})

#Avg titer with ZOC
avg_ZOC_titer = ZOC_no_cock.groupby("Strain")["result"].agg(["mean", "std"]).reset_index()
avg_ZOC_titer.columns = ["Strain", "Avg_Titer_wZOC", "StDev_wZOC"]

sorted_ZOC_titers = avg_ZOC_titer.sort_values(by="Avg_Titer_wZOC")

all_ZOC_plaquing = pd.merge(sorted_ZOC_titers, ZOCmax_whatmatters, on="Strain", how="left")

#convert to scientific notation
# columns_to_convert = ['Avg_Titer', 'StDev']
# sorted_ZOC_titers[columns_to_convert] = sorted_ZOC_titers[columns_to_convert].apply(lambda x: ['{:e}'.format(val) for val in x])
# sorted_ZOC_titers.to_csv("ClinIsolates_avg_ZOC_Titers.csv", index=False)

### Merge with Non-ZOC Avgs and EOPs

In [ ]:
last_columns_to_convert = ['Avg_Titer', 'StDev_Titer', 'max_titer', "Avg_Titer_wZOC", "StDev_wZOC", 'max_titer_wZOC']

ClinIsos_ZOCspots = pd.merge(ClinIsos_spots, all_ZOC_plaquing, on="Strain", how="left")
ClinIsos_ZOCspots = ClinIsos_ZOCspots.sort_values(by="Avg_Titer_wZOC")
ClinIsos_ZOCspots[last_columns_to_convert] = ClinIsos_ZOCspots[last_columns_to_convert].astype(float)
ClinIsos_ZOCspots[last_columns_to_convert] = ClinIsos_ZOCspots[last_columns_to_convert].apply(lambda x: ['{:e}'.format(val) for val in x])
ClinIsos_ZOCspots.to_csv(#redacted, index=False)

unique_isos_ZOCspots = pd.merge(unique_isos, ClinIsos_ZOCspots, on="Strain", how="left")
unique_isos_ZOCspots[last_columns_to_convert] = unique_isos_ZOCspots[last_columns_to_convert].astype(float)
unique_isos_ZOCspots[last_columns_to_convert] = unique_isos_ZOCspots[last_columns_to_convert].apply(lambda x: ['{:e}'.format(val) for val in x])
unique_isos_ZOCspots.to_csv(#redacted, index=False)

baseline_isos_ZOCspots = pd.merge(baseline_isos, ClinIsos_ZOCspots, on=["Strain"], how="left")
baseline_isos_ZOCspots[last_columns_to_convert] = baseline_isos_ZOCspots[last_columns_to_convert].astype(float)
baseline_isos_ZOCspots[last_columns_to_convert] = baseline_isos_ZOCspots[last_columns_to_convert].apply(lambda x: ['{:e}'.format(val) for val in x])
baseline_isos_ZOCspots.to_csv(#redacted, index=False)